In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
os.chdir('drive/nnfl/NNFL/')

In [15]:
!ls

datalab  drive


In [2]:
import numpy as np
import keras
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,Dropout, Reshape,Concatenate
from keras.models import Model, load_model,Sequential

Using TensorFlow backend.


In [0]:
resnet = keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet' ,input_shape=(224,224,3))


In [0]:
for layer in resnet.layers:
  layer.trainable = True

Resnet+tranfer from imagenet+siamese

In [52]:
X_in_1 = Input(shape=(224,224,3))
X_in_2 = Input(shape=(224,224,3))
X_in_3 = Input(shape=(224,224,3))

X1 = resnet(X_in_1)
X2 = resnet(X_in_2)
X3 = resnet(X_in_3)
#X = Dense(12,activation='softmax')(X)
X = Concatenate(axis=1)([X1,X2,X3])
model = Model(inputs=[X_in_1,X_in_2,X_in_3],outputs=X)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
resnet50 (Model)                (None, 1, 1, 2048)   23587712    input_9[0][0]                    
                                                                 input_10[0][0]                   
          

In [0]:
model.summary()

In [0]:
import tensorflow as tf
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[:,0,:], y_pred[:,1,:], y_pred[:,2,:]
    
    ### START CODE HERE ### (≈ 4 lines)
    # Step 1: Compute the (encoding) distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(anchor - positive))
    # Step 2: Compute the (encoding) distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(anchor - negative))
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = pos_dist - neg_dist + alpha
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.maximum(basic_loss, 0)
    ### END CODE HERE ###
    
    return loss

In [0]:
model.compile(optimizer='adam',loss=triplet_loss,metrics=["accuracy"])

In [0]:
import h5py
hf=h5py.File('data_x.h5','r')
x = hf.get('dataset_1')
y = hf.get('dataset_2')

x = np.asarray(x)
y = np.asarray(y)

In [0]:
import cv2
import random

In [61]:
import os
path = os.listdir('.')
path.sort(key=str)
fold = path[:12]
#print(fold)

for j in range(100):
  X1 = []
  X2 = []
  X3 = []
  for i in range(50):
      a,b = random.sample(range(len(fold)),2)
      fold1 = os.listdir(fold[a])
      fold1.sort(key=str.lower)
      fold2 = os.listdir(fold[b])
      fold2.sort(key=str.lower)
      #print(len(fold1))
      c,d = random.sample(range(0,len(fold1[:-1])),2)
      e = np.random.randint(0,len(fold2[:-1]))
      #print(c==d)
      img1 = np.asarray(cv2.imread(fold[a]+'/'+fold1[c],1))
      ran = np.random.randint(0,100)
      img1 = img1[ran:ran+224,ran:ran+224] 
      img2 = np.asarray(cv2.imread(fold[a]+'/'+fold1[d],1))
      ran = np.random.randint(0,100)
      img2 = img2[ran:ran+224,ran:ran+224]
      img3 = np.asarray(cv2.imread(fold[b]+'/'+fold2[e],1))
      ran = np.random.randint(0,100)
      img3 = img3[ran:ran+224,ran:ran+224]
      #print(img1.shape,img2.shape,img3.shape)
      #print(img1.all() == img2.all())
      if(len(img1.shape) == 3 and len(img2.shape) == 3 and len(img3.shape) == 3):
        X1.append(img1)
        X2.append(img2)  
        X3.append(img3)
        
  X1 = np.asarray(X1)
  print(X1.shape)
  X2 = np.asarray(X2)
  X3 = np.asarray(X3)
  y = np.zeros((X1.shape[0],3,1,2048))
  print(y.shape)
  model.fit([X1,X2,X3],y,epochs=5,batch_size=8)
  resnet.save('siam_resnet.h5')

(50, 224, 224, 3)
(50, 3, 1, 2048)
Epoch 1/5
50/50 [==============================] - 7s 134ms/step - loss: 155.2262 - acc: 0.0000e+00
Epoch 2/5
50/50 [==============================] - 7s 130ms/step - loss: 137.5706 - acc: 0.0000e+00
Epoch 3/5
50/50 [==============================] - 7s 131ms/step - loss: 1821.9678 - acc: 0.0000e+00
Epoch 4/5
50/50 [==============================] - 6s 130ms/step - loss: 3352.9157 - acc: 0.0000e+00
Epoch 5/5
50/50 [==============================] - 7s 130ms/step - loss: 834.3945 - acc: 0.0000e+00
(50, 224, 224, 3)
(50, 3, 1, 2048)
Epoch 1/5
50/50 [==============================] - 7s 134ms/step - loss: 114.6438 - acc: 0.0000e+00
Epoch 2/5
50/50 [==============================] - 7s 130ms/step - loss: 161.7793 - acc: 0.0000e+00
Epoch 3/5
50/50 [==============================] - 7s 131ms/step - loss: 85.0573 - acc: 0.0000e+00
Epoch 4/5
50/50 [==============================] - 7s 131ms/step - loss: 34.3434 - acc: 0.0000e+00
Epoch 5/5
50/50 [=============

ValueError: ignored